Read Data

In [46]:
import pandas as pd
import numpy as np
from statistics import mode
import math
from openpyxl import load_workbook
import time

def read_excel(path, sheet_target): # read excel data
    global category
    data = pd.read_excel(path, sheet_name=sheet_target)
    return pd.DataFrame(data)

def reScale(data, method):
    global category
    if(method == 'std'):
        for i in category:
            data[i] = (data[i] - data[i].mean()) / data[i].std()
    elif(method == 'norm'):
        for i in category:
            data[i] = (data[i] - data[i].min())/data[i].std()
    
    return data

category = ['x1', 'x2', 'x3']
data = read_excel('traintest.xlsx', 'train')

Describing the data

In [47]:
data.describe()

,id,x1,x2,x3,y
count,296.000000,296.000000,296.000000,296.000000,296.000000
mean,148.500000,52.462838,62.881757,4.111486,0.736486
std,85.592056,10.896367,3.233753,7.291816,0.441285
min,1.000000,30.000000,58.000000,0.000000,0.000000
25%,74.750000,44.000000,60.000000,0.000000,0.000000
50%,148.500000,52.000000,63.000000,1.000000,1.000000
75%,222.250000,61.000000,65.250000,5.000000,1.000000
max,296.000000,83.000000,69.000000,52.000000,1.000000


In [48]:
data['y'].value_counts()

1    218
0     78
Name: y, dtype: int64

We use standardization to make our algorithm easier to find the neighbor

In [49]:
reScaled = reScale(data, 'std')
reScaled

,id,x1,x2,x3,y
0,1,0.691713,0.345804,-0.563849,1
1,2,0.141071,-0.891149,0.944691,0
2,3,1.150582,-0.272673,2.453232,0
3,4,-1.694403,-0.891149,-0.563849,1
4,5,-1.327308,1.891995,2.316092,0
...,...,...,...,...,...
291,292,0.599940,0.345804,-0.426709,1
292,293,1.150582,1.273518,-0.563849,1
293,294,0.049297,0.655042,1.081831,0
294,295,0.416392,0.345804,-0.426709,0


Data describing after we do standardization scaling

Cross-Validation based on index

In [50]:
def cross_data(train_data, total):
    fold = train_data.shape[0]//total
    split_loc = []
    a = 0
    b = fold-1
    for i in range(total):
        split_loc.append([a,b])
        a += fold
        b += fold
    return split_loc, fold

Find distance with Euclidean

In [51]:
def euclidean(X_train, X_test, i, j):
    #a = np.array([X_train['x1'][j], X_train['x2'][j]])
    #b = np.array([X_test['x1'][i], X_test['x2'][i]])
    a = np.array([X_train['x1'][j], X_train['x2'][j],X_train['x3'][j]])
    b = np.array([X_test['x1'][i], X_test['x2'][i],X_test['x3'][i]])
    eu_dist = np.linalg.norm(b-a)
    return eu_dist

Find distance with Manhattan

In [52]:
def manhattanDist(X_train, X_test, i, j):
    a = np.array([X_train['x1'][j], X_train['x2'][j],X_train['x3'][j]])
    b = np.array([X_test['x1'][i], X_test['x2'][i],X_test['x3'][i]])
    eu_dist = np.abs(a[:None]- b).sum(-1)
    return abs(eu_dist)

Find distance with Minkowski

In [53]:
def minkowski(X_train, X_test, i, j, p = 3): # test or X_test
    #a = np.array(X_test[category[0]][i],X_test[category[1]][i])
    #b = np.array(X_train[category[0]][j],X_train[category[1]][j])
    a = np.array([X_train['x1'][j], X_train['x2'][j],X_train['x3'][j]])
    b = np.array([X_test['x1'][i], X_test['x2'][i],X_test['x3'][i]])

    eu_dist = np.abs(a[:None]- b).sum(-1) ** p
    return eu_dist ** (1/p)

Find distance with Supremum

In [54]:
def supremum(X_train, X_test, i, j): # test or X_test
    train = np.array([X_train['x1'][j], X_train['x2'][j],X_train['x3'][j]])
    test = np.array([X_test['x1'][i], X_test['x2'][i],X_test['x3'][i]])

    return max([abs(train[l] - test[l]) for l in range(len(train))])

Find the closest neighbor

In [55]:
def closest_neighbor(train_data, validate_data,k, method, debug):
    closest = []
    for i in range(len(validate_data)):
        temp = []
        for j in range(len(train_data)):
            if(method == 'eu'):
                temp.append([train_data['id'][j], euclidean(train_data, validate_data, i, j)])
            elif(method == 'manhat'):
                temp.append([train_data['id'][j], manhattanDist(train_data, validate_data, i, j)])
            elif(method == 'mink'):
                temp.append([train_data['id'][j], minkowski(train_data, validate_data, i, j)])
            elif(method == 'supre'):
                temp.append([train_data['id'][j], supremum(train_data, validate_data, i, j)])
        temp = sorted(temp, key=lambda l: l[1])[0:k]
        closest.append({'vali_id': validate_data['id'][i], 'closest': list(map(lambda c : c[0], [c for c in temp]))})
    if(debug):
        print(closest)
    return closest

kNN algorithm

In [56]:
def kNN(train_data, validate_data, k, method, debug, run_as):
    neighbor = closest_neighbor(train_data, validate_data,k, method, debug)
    prediction_arr = []
    for i in range(len(validate_data)):
        truth_value = []
        for j in neighbor[i]['closest']:
            truth_value.append(int(train_data.iloc[np.where(train_data['id'] == j)]['y']))
        prediction_arr.append(mode(truth_value))
    if(run_as == 'train'):
        validate_data['prediction'] = prediction_arr
    elif(run_as == 'predict'):
        validate_data['y'] = prediction_arr
        if(debug):
            print(validate_data)

    return validate_data
            

Confussion matrix to evaluate the model (k value)

In [57]:
def confussion_matrix(trained):
    tp, fp, fn, tn = 0,0,0,0
    for i in range(trained.shape[0]):
        if((trained['y'][i] == 1) and (trained['prediction'][i] == 1)): tp += 1
        elif((trained['y'][i] == 0) and (trained['prediction'][i] == 1)): fp += 1
        elif((trained['y'][i] == 1) and (trained['prediction'][i] == 0)): fn += 1
        elif((trained['y'][i] == 0) and (trained['prediction'][i] == 0)): tn += 1
    
    l = 10**(-10)
    acc =  ((tp+tn)/(tp+fp+tn+fn+l))*100
    prec = (tp/(tp+fp+l))*100
    spec = (tn/(tn+fp+l))*100
    recc = (tp/(tp+fn+l))*100
    f1 = 2*((prec*recc)/(prec+recc+l))
    miss = fn / (fn + tp+l)
    mathew = ((tp*tn) - (fp*fn)) / math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)+l)
    acc_ball = (recc+spec)/2
    return {'Accuracy' : acc, 'Precision' : prec, 'Specificity' : spec, 'Recall' : recc, 'F1 Score':f1, 'Miss Rate':miss, 'Accuracy Balance':acc_ball, 'Mathew Correlation': mathew,'Matrix':(tp,fp,fn,tn)}

Training driver

In [58]:
def training_model(data, method, debug, run_as, reScale_method, fold_total):
    data = reScale(data, reScale_method)
    training_conclusion = []
    acc_array = []
    split_loc, fold = cross_data(data, fold_total)
    for i in split_loc:
        acc_array = []
        validate_data = data[i[0]:i[1]]
        train_data = data.drop(data.index[i[0]:i[1]])
        for k in range(3, fold, 2):    
            trained = kNN(train_data.reset_index(), validate_data.reset_index(), k, method, debug, run_as)
            cof_matrix = confussion_matrix(trained)
            acc_array.append([k, cof_matrix['Accuracy'],cof_matrix])
        training_conclusion.append(acc_array)
    return (training_conclusion, split_loc)

Find the most optimal k

> based on k alone (not used)

In [59]:
def k_optimal(model_conclusion, split_index):
    optimal_k = []
    for i in range(len(model_conclusion)):
        optimal_k.append(max(model_conclusion[i], key=lambda l: l[1])[0])

    modus = mode(optimal_k)
    acc =  0
    prec = 0
    spec = 0
    recc = 0
    f1 = 0
    miss = 0
    mathew = 0
    acc_ball = 0
    tp, fp, fn, tn = 0,0,0,0
    n = 0
    for i in model_conclusion:
        for j in i:
            if(j[0] == modus):
                acc += j[2]['Accuracy']
                prec += j[2]['Precision']
                spec += j[2]['Specificity']
                recc += j[2]['Recall']
                f1 += j[2]['F1 Score']
                miss += j[2]['Miss Rate']
                mathew += j[2]['Mathew Correlation']
                acc_ball += j[2]['Accuracy Balance']
                tp += j[2]['Matrix'][0]
                fp += j[2]['Matrix'][1]
                fn += j[2]['Matrix'][2]
                tn += j[2]['Matrix'][3]
                n += 1
    
    model_report = {'Accuracy' : acc/n, 'Precision' : prec/n, 'Specificity' : spec/n, 'Recall' : recc/n, 'F1 Score':f1/n, 'Miss Rate':miss/n, 'Accuracy Balance':acc_ball/n, 'Mathew Correlation': mathew/n,'Matrix':(tp/n,fp/n,fn/n,tn/n)}
    best_index = [0,0]
    for x in range(len(model_conclusion)):
        for y in range(len(model_conclusion[x])):
            if((model_conclusion[x][y][0] == modus) and (model_conclusion[x][y][1] > model_conclusion[best_index[0]][best_index[1]][1])):
                best_index = [x,y]

    return modus, model_report, split_index[best_index[0]]

> based on the model and its k

In [60]:
def model_optimal(model_conclusion, split_index):
    c = []
    for j in range(len(split_index)):
        a = max(model_conclusion[j], key=lambda l: l[1])[0]
        loc = [i for i, t in enumerate(model_conclusion[j]) if t[0]==a]
        c.append([j,loc[0]])
    
    high_index = c[0]
    for x in c:
        if(model_conclusion[x[0]][x[1]][1] >= model_conclusion[high_index[0]][high_index[1]][1]):
            high_index = x

    return (split_index[high_index[0]], model_conclusion[high_index[0]][high_index[1]][0], model_conclusion[high_index[0]][high_index[1]][2])

Predict the 'test' dataset

In [61]:
def test_prediction(data, test_data, k_value, method, debug, run_as, reScale_method, model_index ,via):
    data = reScale(data, reScale_method)
    data = data.drop(data.index[model_index[0]:model_index[1]])
    data = data.reset_index()
    print(data.shape[0])
    test_data = reScale(test_data, reScale_method)
    prediction = kNN(data, test_data, k_value, method, debug, run_as)
    return prediction

In [62]:
def model_report(knn_method, optimal_k, reScale_method, fold_total, end_time, start_time, model_grade, model_index, via):
    txt = 'Best'
    if(via == 'k_only'):
        txt = 'Average'
    print(f'<Model configuration>\nAlgorithm : KNN\nDistance Method : {knn_method}\n{txt} K Value : {optimal_k}\nScaling Method : {reScale_method}\nK-Fold Cross-Validation : {fold_total} folds')
    print(f"Model Optimizer : {via}\nBest Fold Index : {model_index}\nTraining Time : {end_time-start_time}s")
    print("Model Quality :")
    for i in model_grade:
        if(i == 'Matrix'):
            print('<-> Confussion Matrix')
            print(f'    {model_grade[i][0]}\t{model_grade[i][1]}')
            print(f'    {model_grade[i][2]}\t{model_grade[i][3]}')
        else:
            print(f'<-> {i} {txt} : {model_grade[i]}')

Run the model training

In [63]:
data = read_excel('traintest.xlsx', 'train') 
reScale_method = 'std' 
fold_total = 9
knn_method = 'eu' # eu supre manhat mink
start_time = time.time()
model_conclusion, split_index = training_model(data, knn_method, False, 'train', reScale_method, fold_total)        

In [64]:
via = 'k_only' # k_only model
if(via == 'model'):
    model_index, model_k, model_grade = model_optimal(model_conclusion, split_index)
elif(via == 'k_only'):
    model_k, model_grade, model_index = k_optimal(model_conclusion, split_index) 
end_time = time.time()

print("<<Showing model candidate>>")
for k in range(len(split_index)):
    print(f'\n<Model {k+1}>\nFolding Index : {split_index[k]}')
    for l in range(len(model_conclusion[k])):
        print(f' {model_conclusion[k][l]}')

<<Showing model candidate>>

<Model 1>
Folding Index : [0, 31]
 [3, 77.41935483845994, {'Accuracy': 77.41935483845994, 'Precision': 82.60869565181476, 'Specificity': 55.55555555493827, 'Recall': 86.36363636324381, 'F1 Score': 84.44444444401918, 'Miss Rate': 0.13636363636301654, 'Accuracy Balance': 70.95959595909105, 'Mathew Correlation': 0.434847107644502, 'Matrix': (19, 4, 3, 5)}]
 [5, 70.96774193525495, {'Accuracy': 70.96774193525495, 'Precision': 78.26086956487714, 'Specificity': 44.444444443950616, 'Recall': 81.81818181780991, 'F1 Score': 79.99999999959448, 'Miss Rate': 0.18181818181735537, 'Accuracy Balance': 63.131313130880265, 'Mathew Correlation': 0.27243433250016996, 'Matrix': (18, 5, 4, 4)}]
 [7, 67.74193548365245, {'Accuracy': 67.74193548365245, 'Precision': 73.07692307664202, 'Specificity': 22.222222221975308, 'Recall': 86.36363636324381, 'F1 Score': 79.16666666628717, 'Miss Rate': 0.13636363636301654, 'Accuracy Balance': 54.29292929260956, 'Mathew Correlation': 0.105960658

Model Report

In [65]:
model_report(knn_method, model_k, reScale_method, fold_total, end_time, start_time, model_grade, model_index, via)

<Model configuration>
Algorithm : KNN
Distance Method : eu
Best K Value : 11
Scaling Method : std
K-Fold Cross-Validation : 9 folds
Model Optimizer : model
Best Fold Index : [64, 95]
Training Time : 31.048578023910522s
Model Quality :
<-> Accuracy Best : 90.32258064486993
<-> Precision Best : 89.28571428539541
<-> Specificity Best : 49.99999999916667
<-> Recall Best : 99.9999999996
<-> F1 Score Best : 94.3396226411036
<-> Miss Rate Best : 0.0
<-> Accuracy Balance Best : 74.99999999938333
<-> Mathew Correlation Best : 0.6681531047810583
<-> Confussion Matrix
    25	3
    0	3


Run the prediction

In [66]:
data = read_excel('traintest.xlsx', 'train')
test_data = read_excel('traintest.xlsx', 'test')
if(via=='k_only'):
    print('Total dataset : ', end="")
    prediction = test_prediction(data, test_data, model_k, 'eu', False, 'predict', reScale_method, model_index ,via)
elif(via=='model'):
    print('Total dataset : ', end="")
    prediction = test_prediction(data, test_data, model_k, 'eu', False, 'predict', reScale_method, model_index ,via)

Total dataset : 265


Prediction Result

In [67]:
test_data = read_excel('traintest.xlsx', 'test')
prediction['x1'] = test_data['x1']
prediction['x2'] = test_data['x2']
prediction['x3'] = test_data['x3']
prediction

,id,x1,x2,x3,y
0,297,43,59,2,1
1,298,67,66,0,1
2,299,58,60,3,0
3,300,49,63,3,0
4,301,45,60,0,1
5,302,54,58,1,1
6,303,56,66,3,0
7,304,42,69,1,1
8,305,50,59,2,1
9,306,59,60,0,1


In [68]:
def output_xlsx(prediction):
    book = load_workbook('traintest.xlsx')
    writer = pd.ExcelWriter('traintest.xlsx')
    writer.book = book
    prediction.to_excel(writer, sheet_name= 'test_prediction')
    writer.save()
    writer.close()

output_xlsx(prediction)